In [15]:
import ssnmf as ssnmf
from ssnmf import SSNMF
import numpy as np
import scipy
import scipy.sparse as sparse
import copy
import random
import time

In [16]:
#trial info
M = 1 ################################# just a test
#M = 10  #number of trials
N = 10 ################################ just a test
#N = 1000 #max number of iterations

seedval = 777
#keep track of elapsed time
start = time.time()

In [17]:
#generate factor matrices (these are latent signal throughout all experiments)
n1 = 500
n2 = 500
r = 5
k = 500
random.seed(seedval)
S = sparse.random(r,n2,density=0.5).toarray()
np.random.seed(seedval)
A = np.random.rand(n1,r)
random.seed(seedval)
B = sparse.random(k,r,density=0.5).toarray()

In [18]:
#initialize results arrays
results1 = np.zeros(4) #experiment 1
results2 = np.zeros(4) #experiment 2
results3 = np.zeros(4) #experiment 3
results4 = np.zeros(4) #experiment 4

# Gaussian Uncertainty

In [19]:
#generate X and Y with Gaussian uncertainty (as determined by lambda in paper)
Xmean = A @ S
lam=1
Ymean = B @ S
np.random.seed(seedval)
X = abs(Xmean + np.random.normal(0,1,(n1,n2)))
np.random.seed(seedval)
Y = abs(Ymean + np.random.normal(0,np.sqrt(1/lam),(k,n2)))

In [20]:
#initialize results
relerrs3 = np.zeros(M)
relerrs4 = np.zeros(M)
relerrs5 = np.zeros(M)
relerrs6 = np.zeros(M)

In [21]:
#run trials of SSNMF models
for i in range(M):
    #generate (seeded) initial iterates for all methods
    np.random.seed(i)
    A0 = np.random.rand(n1, r)
    np.random.seed(i+M)
    B0 = np.random.rand(k,r)
    np.random.seed(i+2*M)
    S0 = np.random.rand(r, n2)
    
    #initialize model (we make copies for each SSNMF model)
    model = SSNMF(X,r,Y = Y,lam=lam,tol=1e-14,A=A0,B=B0,S=S0)
    X0 = model.A @ model.S
    Y0 = model.B @ model.S
    
    #copy model for each type of SSNMF and run multiplicative updates
    model_3 = copy.deepcopy(model)
    model_3.modelNum = 3
    model_3.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_4 = copy.deepcopy(model)
    model_4.modelNum = 4
    model_4.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_5 = copy.deepcopy(model)
    model_5.modelNum = 5
    model_5.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_6 = copy.deepcopy(model)
    model_6.modelNum = 6
    model_6.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    #collect metrics
    relerrs3[i] = (np.linalg.norm(Xmean - model_3.A @ model_3.S)**2 + lam*np.linalg.norm(Ymean - model_3.B @ model_3.S)**2)/(np.linalg.norm(Xmean - X0)**2 + lam*np.linalg.norm(Ymean - Y0)**2)
    relerrs4[i] = (np.linalg.norm(Xmean - model_4.A @ model_4.S)**2 + lam*np.linalg.norm(Ymean - model_4.B @ model_4.S)**2)/(np.linalg.norm(Xmean - X0)**2 + lam*np.linalg.norm(Ymean - Y0)**2)
    relerrs5[i] = (np.linalg.norm(Xmean - model_5.A @ model_5.S)**2 + lam*np.linalg.norm(Ymean - model_5.B @ model_5.S)**2)/(np.linalg.norm(Xmean - X0)**2 + lam*np.linalg.norm(Ymean - Y0)**2)
    relerrs6[i] = (np.linalg.norm(Xmean - model_6.A @ model_6.S)**2 + lam*np.linalg.norm(Ymean - model_6.B @ model_6.S)**2)/(np.linalg.norm(Xmean - X0)**2 + lam*np.linalg.norm(Ymean - Y0)**2)

In [22]:
#save results (mean of relative errors over trials)
results1[0] = np.mean(relerrs3)
results1[1] = np.mean(relerrs4)
results1[2] = np.mean(relerrs5)
results1[3] = np.mean(relerrs6)

# Gaussian and Poisson Uncertainty

In [23]:
#generate X and Y with Gaussian and Poisson uncertainty (as determined by lambda in paper)
Xmean = A @ S
lam=1
Ymean = B @ S
np.random.seed(seedval)
X = abs(Xmean + np.random.normal(0,lam/(2*k),(n1,n2)))
np.random.seed(seedval)
Y = np.random.poisson(Ymean,np.shape(Ymean))

In [24]:
#initialize results
relerrs3 = np.zeros(M)
relerrs4 = np.zeros(M)
relerrs5 = np.zeros(M)
relerrs6 = np.zeros(M)

In [25]:
#run trials of SSNMF models
for i in range(M):
    #generate (seeded) initial iterates for all methods
    np.random.seed(i)
    A0 = np.random.rand(n1, r)
    np.random.seed(i+M)
    B0 = np.random.rand(k,r)
    np.random.seed(i+2*M)
    S0 = np.random.rand(r, n2)
    
    #initialize model (we make copies for each SSNMF model)
    model = SSNMF(X,r,Y = Y,lam=lam,tol=1e-14,A=A0,B=B0,S=S0)
    X0 = model.A @ model.S
    Y0 = model.B @ model.S
    
    #copy model for each type of SSNMF and run multiplicative updates
    model_3 = copy.deepcopy(model)
    model_3.modelNum = 3
    model_3.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_4 = copy.deepcopy(model)
    model_4.modelNum = 4
    model_4.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_5 = copy.deepcopy(model)
    model_5.modelNum = 5
    model_5.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_6 = copy.deepcopy(model)
    model_6.modelNum = 6
    model_6.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    #collect metrics
    relerrs3[i] = (np.linalg.norm(Xmean - model_3.A @ model_3.S)**2 + lam*model_3.Idiv(Ymean, model_3.B,model_3.S,None))/(np.linalg.norm(Xmean - X0)**2 + lam*model_3.Idiv(Ymean, B0,S0,None))
    relerrs4[i] = (np.linalg.norm(Xmean - model_4.A @ model_4.S)**2 + lam*model_4.Idiv(Ymean, model_4.B,model_4.S,None))/(np.linalg.norm(Xmean - X0)**2 + lam*model_4.Idiv(Ymean, B0,S0,None))
    relerrs5[i] = (np.linalg.norm(Xmean - model_5.A @ model_5.S)**2 + lam*model_5.Idiv(Ymean, model_5.B,model_5.S,None))/(np.linalg.norm(Xmean - X0)**2 + lam*model_5.Idiv(Ymean, B0,S0,None))
    relerrs6[i] = (np.linalg.norm(Xmean - model_6.A @ model_6.S)**2 + lam*model_6.Idiv(Ymean, model_6.B,model_6.S,None))/(np.linalg.norm(Xmean - X0)**2 + lam*model_6.Idiv(Ymean, B0,S0,None))

In [26]:
#save results (mean of relative errors over trials)
results2[0] = np.mean(relerrs3)
results2[1] = np.mean(relerrs4)
results2[2] = np.mean(relerrs5)
results2[3] = np.mean(relerrs6)

# Poisson and Gaussian Uncertainty

In [27]:
#generate X and Y with Poisson and Gaussian uncertainty (as determined by lambda in paper)
Xmean = A @ S
lam=1
Ymean = B @ S
np.random.seed(seedval)
Y = abs(Ymean + np.random.normal(0,1/(2*r*lam),(k,n2)))
np.random.seed(seedval)
X = np.random.poisson(Xmean,np.shape(Xmean))

In [28]:
#initialize results
relerrs3 = np.zeros(M)
relerrs4 = np.zeros(M)
relerrs5 = np.zeros(M)
relerrs6 = np.zeros(M)

In [29]:
#run trials of SSNMF models
for i in range(M):
    #generate (seeded) initial iterates for all methods
    np.random.seed(i)
    A0 = np.random.rand(n1, r)
    np.random.seed(i+M)
    B0 = np.random.rand(k,r)
    np.random.seed(i+2*M)
    S0 = np.random.rand(r, n2)
    
    #initialize model (we make copies for each SSNMF model)
    model = SSNMF(X,r,Y = Y,lam=lam,tol=1e-10,A=A0,B=B0,S=S0)
    X0 = model.A @ model.S
    Y0 = model.B @ model.S
    
    #copy model for each type of SSNMF and run multiplicative updates
    model_3 = copy.deepcopy(model)
    model_3.modelNum = 3
    model_3.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_4 = copy.deepcopy(model)
    model_4.modelNum = 4
    model_4.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_5 = copy.deepcopy(model)
    model_5.modelNum = 5
    model_5.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_6 = copy.deepcopy(model)
    model_6.modelNum = 6
    model_6.mult(numiters = N,saveerrs = False, eps = 1e-14)
        
    #collect metrics
    relerrs3[i] = (model_3.Idiv(Xmean,model_3.A,model_3.S,None) + lam*np.linalg.norm(Ymean - model_3.B @model_3.S)**2)/(model_3.Idiv(Xmean,A0,S0,None) + lam*np.linalg.norm(Ymean-Y0)**2)
    relerrs4[i] = (model_4.Idiv(Xmean,model_4.A,model_4.S,None) + lam*np.linalg.norm(Ymean - model_4.B @model_4.S)**2)/(model_4.Idiv(Xmean,A0,S0,None) + lam*np.linalg.norm(Ymean-Y0)**2)
    relerrs5[i] = (model_5.Idiv(Xmean,model_5.A,model_5.S,None) + lam*np.linalg.norm(Ymean - model_5.B @model_5.S)**2)/(model_5.Idiv(Xmean,A0,S0,None) + lam*np.linalg.norm(Ymean-Y0)**2)
    relerrs6[i] = (model_6.Idiv(Xmean,model_6.A,model_6.S,None) + lam*np.linalg.norm(Ymean - model_6.B @model_6.S)**2)/(model_6.Idiv(Xmean,A0,S0,None) + lam*np.linalg.norm(Ymean-Y0)**2)

In [30]:
#save results (mean of relative errors over trials)
results3[0] = np.mean(relerrs3)
results3[1] = np.mean(relerrs4)
results3[2] = np.mean(relerrs5)
results3[3] = np.mean(relerrs6)

# Poisson Uncertainty

In [31]:
#generate X and Y with Poisson uncertainty (as determined in paper, lambda = 1)
Xmean = A @ S
lam=1
Ymean = B @ S
np.random.seed(seedval)
Y = np.random.poisson(Ymean,np.shape(Ymean))
np.random.seed(seedval)
X = np.random.poisson(Xmean,np.shape(Xmean))

In [32]:
#initialize results
relerrs3 = np.zeros(M)
relerrs4 = np.zeros(M)
relerrs5 = np.zeros(M)
relerrs6 = np.zeros(M)

In [33]:
#run trials of SSNMF models
for i in range(M):
    #generate (seeded) initial iterates for all methods
    np.random.seed(i)
    A0 = np.random.rand(n1, r)
    np.random.seed(i+M)
    B0 = np.random.rand(k,r)
    np.random.seed(i+2*M)
    S0 = np.random.rand(r, n2)
    
    #initialize model (we make copies for each SSNMF model)
    model = SSNMF(X,r,Y = Y,lam=lam,tol=1e-14,A=A0,B=B0,S=S0)
    X0 = model.A @ model.S
    Y0 = model.B @ model.S
    
    #copy model for each type of SSNMF and run multiplicative updates
    model_3 = copy.deepcopy(model)
    model_3.modelNum = 3
    model_3.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_4 = copy.deepcopy(model)
    model_4.modelNum = 4
    model_4.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_5 = copy.deepcopy(model)
    model_5.modelNum = 5
    model_5.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    model_6 = copy.deepcopy(model)
    model_6.modelNum = 6
    model_6.mult(numiters = N,saveerrs = False, eps = 1e-14)
    
    #collect metrics
    relerrs3[i] = (model_3.Idiv(Xmean,model_3.A,model_3.S,None) + lam*model_3.Idiv(Ymean,model_3.B,model_3.S,None))/(model_3.Idiv(Xmean,A0,S0,None) + lam*model_3.Idiv(Ymean,B0,S0,None))
    relerrs4[i] = (model_4.Idiv(Xmean,model_4.A,model_4.S,None) + lam*model_4.Idiv(Ymean,model_4.B,model_4.S,None))/(model_4.Idiv(Xmean,A0,S0,None) + lam*model_4.Idiv(Ymean,B0,S0,None))
    relerrs5[i] = (model_5.Idiv(Xmean,model_5.A,model_5.S,None) + lam*model_5.Idiv(Ymean,model_5.B,model_5.S,None))/(model_5.Idiv(Xmean,A0,S0,None) + lam*model_5.Idiv(Ymean,B0,S0,None))
    relerrs6[i] = (model_6.Idiv(Xmean,model_6.A,model_6.S,None) + lam*model_6.Idiv(Ymean,model_6.B,model_6.S,None))/(model_6.Idiv(Xmean,A0,S0,None) + lam*model_6.Idiv(Ymean,B0,S0,None))

In [34]:
#save results (mean of relative errors over trials)
results4[0] = np.mean(relerrs3)
results4[1] = np.mean(relerrs4)
results4[2] = np.mean(relerrs5)
results4[3] = np.mean(relerrs6)

# Results

In [35]:
print('{:<20}'.format(''), '{:<20}'.format('Fro-Fro-SSNMF'),'{:<20}'.format('Fro-Div-SSNMF'),'{:<20}'.format('Div-Fro-SSNMF'),'{:<20}'.format('Div-Div-SSNMF'))
print('{:<20}'.format('Experiment 1 (GG): '), '{:<20}'.format(results1[0]),'{:<20}'.format(results1[1]),'{:<20}'.format(results1[2]),'{:<20}'.format(results1[3]))
print('{:<20}'.format('Experiment 2 (GP): '), '{:<20}'.format(results2[0]),'{:<20}'.format(results2[1]),'{:<20}'.format(results2[2]),'{:<20}'.format(results2[3]))
print('{:<20}'.format('Experiment 3 (PG): '), '{:<20}'.format(results3[0]),'{:<20}'.format(results3[1]),'{:<20}'.format(results3[2]),'{:<20}'.format(results3[3]))
print('{:<20}'.format('Experiment 4 (PP): '), '{:<20}'.format(results4[0]),'{:<20}'.format(results4[1]),'{:<20}'.format(results4[2]),'{:<20}'.format(results4[3]))


                     Fro-Fro-SSNMF        Fro-Div-SSNMF        Div-Fro-SSNMF        Div-Div-SSNMF       
Experiment 1 (GG):   0.31040293077704767  0.3143027642983163   0.3186756265456603   0.32060248878961733 
Experiment 2 (GP):   0.06996586215066926  0.06874316079621422  0.06947234525929448  0.06761133378586594 
Experiment 3 (PG):   0.04806862446363786  0.051006507948554734 0.04635371426023707  0.048004615621966354
Experiment 4 (PP):   0.09585677074377905  0.09445644418123003  0.09459483024713347  0.09233338887591122 


In [36]:
end = time.time()
print("Elapsed time for all trials: ", end-start)

Elapsed time for all trials:  15.640723943710327
